###  first selection

- text-generation []
- decoder []

- open-source models [x]
- HF availability [x]
- Task: text generation [x]
- Date []
- architecture: decoder-based only,  []
- model size < SLMs size [x]
  - https://huggingface.co/docs/accelerate/main/en/usage_guides/model_size_estimator?
  - model size from endpoint?
  - safetensors -> model_info.safetensors -> https://huggingface.co/microsoft/phi-2 [x]
  - check it matches our data about models used []

- [?]Code generation [] used in code generation, trained in code datasets?


In [2]:
from huggingface_hub import HfApi
import pandas as pd


In [12]:
pwd

'/home/fjdur/energy-ml-serving'

In [13]:
import sys
sys.path.append("model_selection/") 

In [17]:
# Initialize API
api = HfApi()

# Fetch models metadata
limit = 20
models = api.list_models(task="text-generation", pipeline_tag="text-generation",sort="downloads", limit=limit)
#models = api.list_models(task="text-generation", sort="downloads", limit=500)



In [4]:
import pandas as pd

# Filter for decoder-only architectures
decoder_models = []
i = 0
num_models = 200000

for model in models:
    print(f"{i}-->")
    if i > num_models:
        print("breaking")
        break

    model_id = getattr(model, "modelId", "Unknown")
    print(f'model_id: {model_id}')

    # Fetch detailed model information
    model_info = api.model_info(model.modelId)
    print(f'config: {model_info.config}')
    print(f'd1: {model_info.downloads_all_time}')
    print(f'd2: {model_info.downloads}')
    #print(f'gguf: {model_info.gguf}')
    print(f'safetensors: {model_info.safetensors}')



    if model_info.config ==None:
        continue

    # Collect model metadata with the desired column names
    model_metadata = {
        #"model": model.modelId,  # Model name or ID
        "model": getattr(model, "modelId", "Unknown"),

        "id": getattr(model, "modelId", "Unknown"),
        #"downloads_all_time": getattr(model_info, "downloads_all_time", "Unknown"),
        "downloads": getattr(model_info, "downloads", "Unknown"),
        "safetensors": getattr(model_info, "safetensors", "Unknown"),

        "tags": getattr(model, "tags", "Unknown"),
        "task": getattr(model_info, "pipeline_tag", "Unknown"),
        "card_data": getattr(model_info, "card_data", "Unknown"),
        "created_at": getattr(model, "created_at", "Unknown"),
        "library_name": getattr(model, "library_name", "Unknown"),
        "transformers_info": getattr(model_info, "transformers_info", "Unknown"),
        "architecture": model_info.config.get("architectures", "Unknown")
    }
    print(f'model_metadata: {model_metadata}')

    decoder_models.append(model_metadata)
    i += 1




0-->
model_id: openai-community/gpt2
config: {'architectures': ['GPT2LMHeadModel'], 'model_type': 'gpt2', 'tokenizer_config': {}}


AttributeError: 'ModelInfo' object has no attribute 'downloads_all_time'

In [72]:
# Check for inconsistencies
for index, item in enumerate(decoder_models):
    if not isinstance(item, dict):
        print(f"Item at index {index} is not a dictionary: {item}")
    else:
        # Check for problematic keys or data
        for key, value in item.items():
            if value is None:
                print(f"Key '{key}' in item at index {index} has a value of None.")


Key 'safetensors' in item at index 5 has a value of None.
Key 'safetensors' in item at index 6 has a value of None.


In [5]:
# Attempt to create the DataFrame in a simplified way
try:
    df = pd.DataFrame(decoder_models)
    print("DataFrame created successfully")
    #print(df.head())  # Display only the first few rows to avoid rendering issues
except Exception as e:
    print(f"Error creating DataFrame: {e}")

DataFrame created successfully


In [7]:
df

""


In [75]:
df.to_csv("decoder_models.csv", index=False)
print("CSV file saved: decoder_models.csv")

CSV file saved: decoder_models.csv


In [46]:
df.columns

Index(['model', 'id', 'downloads', 'safetensors', 'tags', 'task', 'created_at',
       'library_name', 'transformers_info', 'architecture'],
      dtype='object')

In [47]:
df['created_at']

0    2024-09-17 14:10:29+00:00
1    2022-03-02 23:29:04+00:00
2    2024-07-16 01:29:54+00:00
3    2024-07-16 08:07:29+00:00
4    2022-10-08 16:14:42+00:00
5    2022-05-11 08:25:17+00:00
6    2024-07-18 08:56:00+00:00
7    2022-03-02 23:29:04+00:00
8    2024-07-16 16:07:46+00:00
9    2024-04-17 09:35:12+00:00
10   2023-02-08 15:12:33+00:00
11   2023-03-29 07:11:13+00:00
12   2024-09-18 15:12:47+00:00
13   2024-07-04 10:15:41+00:00
14   2023-07-13 16:16:13+00:00
15   2024-09-18 15:03:14+00:00
16   2023-12-30 06:27:30+00:00
17   2024-10-22 15:44:28+00:00
18   2023-06-17 19:52:11+00:00
19   2022-03-02 23:29:04+00:00
Name: created_at, dtype: datetime64[ns, UTC]

## Read file

In [18]:
import math
import re

In [20]:
file_name = 'model_selection/results/decoder_models_info_01.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 139013


In [21]:
df.iloc[0]['safetensors']


"SafeTensorsInfo(parameters={'BF16': 1711376384}, total=1711376384)"

In [64]:
# Filter out rows where the 'safetensors' column is NaN
filtered_df = df[df['safetensors'].notna()]

# Display the filtered DataFrame
print(len(filtered_df))

84420


In [65]:
# Function to extract the 'total' value from the 'safetensors' string
def extract_total(safetensors_info):
    if pd.isna(safetensors_info):
        return None
    match = re.search(r"total=(\d+)", safetensors_info)
    return int(match.group(1)) if match else None

# Create a new column 'num_parameters' with the extracted total
filtered_df['num_parameters'] = filtered_df['safetensors'].apply(extract_total)
filtered_df.head()

/tmp/ipykernel_5622/1210555001.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['num_parameters'] = filtered_df['safetensors'].apply(extract_total)


,model,id,created_at,downloads_all_time,downloads,safetensors,task,library_name,transformers_info,architecture,tags,num_parameters
0,HuggingFaceTB/SmolLM2-1.7B-Instruct,HuggingFaceTB/SmolLM2-1.7B-Instruct,2024-10-31 13:42:06+00:00,NaN,5369,SafeTensorsInfo(parameters={'BF16': 1711376384...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'tensorboard', 'onnx', 'safet...",1711376384
1,nvidia/Llama-3.1-Nemotron-70B-Instruct-HF,nvidia/Llama-3.1-Nemotron-70B-Instruct-HF,2024-10-12 02:37:13+00:00,NaN,167648,SafeTensorsInfo(parameters={'BF16': 7055370649...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",70553706496
2,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,NaN,1171141,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",1235814400
3,facebook/MobileLLM-1B,facebook/MobileLLM-1B,2024-10-31 00:07:47+00:00,NaN,1837,SafeTensorsInfo(parameters={'BF16': 1046421760...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",1046421760
4,CohereForAI/aya-expanse-8b,CohereForAI/aya-expanse-8b,2024-10-23 06:34:13+00:00,NaN,14698,SafeTensorsInfo(parameters={'F16': 8028033024}...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['CohereForCausalLM'],"['transformers', 'safetensors', 'cohere', 'tex...",8028033024


In [66]:
filtered_df['created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')

/tmp/ipykernel_5622/2593281030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')


In [67]:
# Assuming 'decoder_models' is already populated with the model metadata
#df = pd.DataFrame(decoder_models)

# Convert 'created_at' to datetime format, handling possible parsing errors
#df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# Filter models with 'created_at' in 2023 or later
year = 2021
filtered_df = filtered_df[filtered_df['created_at'].dt.year >= year]

print(f"filtered w/ created_at len now:{len(filtered_df)}")


filtered w/ created_at len now:84420


In [68]:
# Filter out rows where the 'safetensors' column is NaN
limit_slm_parameters=5000000000 # 10B
filtered_df = filtered_df[filtered_df['num_parameters'] <= limit_slm_parameters]

# Display the filtered DataFrame
print(len(filtered_df))
filtered_df.head()

43445


,model,id,created_at,downloads_all_time,downloads,safetensors,task,library_name,transformers_info,architecture,tags,num_parameters
0,HuggingFaceTB/SmolLM2-1.7B-Instruct,HuggingFaceTB/SmolLM2-1.7B-Instruct,2024-10-31 13:42:06+00:00,NaN,5369,SafeTensorsInfo(parameters={'BF16': 1711376384...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'tensorboard', 'onnx', 'safet...",1711376384
2,meta-llama/Llama-3.2-1B,meta-llama/Llama-3.2-1B,2024-09-18 15:03:14+00:00,NaN,1171141,SafeTensorsInfo(parameters={'BF16': 1235814400...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",1235814400
3,facebook/MobileLLM-1B,facebook/MobileLLM-1B,2024-10-31 00:07:47+00:00,NaN,1837,SafeTensorsInfo(parameters={'BF16': 1046421760...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",1046421760
6,meta-llama/Llama-3.2-3B-Instruct,meta-llama/Llama-3.2-3B-Instruct,2024-09-18 15:19:20+00:00,NaN,992933,SafeTensorsInfo(parameters={'BF16': 3212749824...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",3212749824
8,facebook/MobileLLM-125M,facebook/MobileLLM-125M,2024-10-30 22:48:34+00:00,NaN,2048,"SafeTensorsInfo(parameters={'F16': 124635456},...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MobileLLMForCausalLM'],"['transformers', 'pytorch', 'safetensors', 'mo...",124635456


In [43]:

save_filtered_df =True
if save_filtered_df:
    # Optional: Save the filtered DataFrame to a CSV file
    filtered_df.to_csv("results/filtered_models.csv", index=False)
    print("Filtered CSV file saved: filtered_models.csv")

Filtered CSV file saved: filtered_models.csv


In [45]:
# stratification criteria
import random
i = random.randint(0,len(filtered_df))
print(i)
row = filtered_df.iloc[i]
row

15329


model                                           mehdirafiei/SQLCODER7B2
id                                              mehdirafiei/SQLCODER7B2
created_at                                    2024-02-17 11:08:37+00:00
downloads_all_time                                                  NaN
downloads                                                            12
safetensors           SafeTensorsInfo(parameters={'BF16': 6738546688...
task                                                    text-generation
library_name                                               transformers
transformers_info     TransformersInfo(auto_model='AutoModelForCausa...
architecture                                       ['LlamaForCausalLM']
tags                  ['transformers', 'safetensors', 'llama', 'text...
num_parameters                                               6738546688
Name: 54760, dtype: object

# stratified sampling

In [58]:
file_name = 'results/filtered_models.csv'
df = pd.read_csv(file_name)
print(f'total models: {len(df)}')

total models: 76437


In [59]:
df.columns

Index(['model', 'id', 'created_at', 'downloads_all_time', 'downloads',
       'safetensors', 'task', 'library_name', 'transformers_info',
       'architecture', 'tags', 'num_parameters'],
      dtype='object')

In [60]:
# add likes: popularity = sum of the normalized likes and downloads
stratification_criteria = ['downloads', 'num_parameters','created_at']

In [49]:
import pandas as pd

# Assuming you have loaded your DataFrame 'df'

# Convert 'created_at' to a numerical format for quartile calculation
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['created_at_numeric'] = df['created_at'].astype(int) // 10**9  # Convert to Unix timestamp, seconds

# Create quartiles for stratified sampling
df['downloads_quartile'] = pd.qcut(df['downloads'], q=2, labels=False)  # 2 quartiles
df['created_at_quartile'] = pd.qcut(df['created_at_numeric'], q=2, labels=False)  # 2 quartiles
df['num_parameters_quartile'] = pd.qcut(df['num_parameters'], q=3, labels=False)  # 3 quartiles

# Display the DataFrame with new quartile columns
print(df[['downloads_quartile', 'created_at_quartile', 'num_parameters_quartile']].head())


   downloads_quartile  created_at_quartile  num_parameters_quartile
0                   1                    1                        1
1                   1                    1                        0
2                   1                    1                        0
3                   1                    1                        2
4                   1                    1                        2


In [51]:
# Group by the quartile columns to create strata
strata = df.groupby(['downloads_quartile', 'created_at_quartile', 'num_parameters_quartile'])

# Select one model from each stratum
selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)

# Display the selected models
selected_models

/tmp/ipykernel_5622/497035949.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_models = strata.apply(lambda x: x.sample(1)).reset_index(drop=True)


,model,id,created_at,downloads_all_time,downloads,safetensors,task,library_name,transformers_info,architecture,tags,num_parameters,created_at_numeric,downloads_quartile,created_at_quartile,num_parameters_quartile
0,FounderOfHuggingface/fresh_gpt2_gen_full_dbped...,FounderOfHuggingface/fresh_gpt2_gen_full_dbped...,2024-01-12 09:12:57+00:00,NaN,4,"SafeTensorsInfo(parameters={'F32': 124439808},...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GPT2LMHeadModel'],"['transformers', 'safetensors', 'gpt2', 'text-...",124439808,1705050777,0,0,0
1,ura-hcmut/MixSUra-AWQ,ura-hcmut/MixSUra-AWQ,2024-01-03 16:44:34+00:00,NaN,0,"SafeTensorsInfo(parameters={'I32': 5850267648,...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MixtralForCausalLM'],"['transformers', 'safetensors', 'mixtral', 'te...",6476533760,1704300274,0,0,1
2,danwils/BatakToba-Seallm-7B-v2,danwils/BatakToba-Seallm-7B-v2,2024-02-24 15:30:14+00:00,NaN,2,SafeTensorsInfo(parameters={'BF16': 7375949824...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MistralForCausalLM'],"['transformers', 'safetensors', 'mistral', 'te...",7375949824,1708788614,0,0,2
3,mlx-community/1.5-Pints-16K-v0.1,mlx-community/1.5-Pints-16K-v0.1,2024-10-28 17:23:19+00:00,NaN,7,SafeTensorsInfo(parameters={'F16': 1565886464}...,text-generation,mlx,NaN,['LlamaForCausalLM'],"['mlx', 'safetensors', 'llama', 'text-generati...",1565886464,1730136199,0,1,0
4,Mursel/mistral_7bv03-tr-qlora,Mursel/mistral_7bv03-tr-qlora,2024-07-22 13:21:56+00:00,NaN,7,"SafeTensorsInfo(parameters={'F32': 151264768, ...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MistralForCausalLM'],"['transformers', 'safetensors', 'mistral', 'te...",3909379392,1721654516,0,1,1
5,mergekit-community/mergekit-slerp-vysthui,mergekit-community/mergekit-slerp-vysthui,2024-07-04 22:11:01+00:00,NaN,7,SafeTensorsInfo(parameters={'BF16': 7241732096...,text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['MistralForCausalLM'],"['transformers', 'safetensors', 'mistral', 'te...",7241732096,1720131061,0,1,2
6,berquetR/phi_first_train,berquetR/phi_first_train,2024-04-24 13:13:43+00:00,NaN,16,"SafeTensorsInfo(parameters={'F32': 18876672, '...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['PhiForCausalLM'],"['transformers', 'safetensors', 'phi', 'text-g...",833179280,1713964423,1,0,0
7,TheBloke/Frostwind-10.7B-v1-GPTQ,TheBloke/Frostwind-10.7B-v1-GPTQ,2023-12-20 08:13:21+00:00,NaN,13,"SafeTensorsInfo(parameters={'I32': 1320714240,...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",1667108864,1703060001,1,0,1
8,TheBloke/meditron-70B-GPTQ,TheBloke/meditron-70B-GPTQ,2023-11-30 17:10:33+00:00,NaN,22,"SafeTensorsInfo(parameters={'I32': 8563445760,...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['LlamaForCausalLM'],"['transformers', 'safetensors', 'llama', 'text...",9102487552,1701364233,1,0,2
9,KANISHKVIJAY/gpt2_interview_model,KANISHKVIJAY/gpt2_interview_model,2024-06-16 12:46:56+00:00,NaN,11,"SafeTensorsInfo(parameters={'F32': 124439808},...",text-generation,transformers,TransformersInfo(auto_model='AutoModelForCausa...,['GPT2LMHeadModel'],"['transformers', 'safetensors', 'gpt2', 'text-...",124439808,1718542016,1,1,0


In [57]:
import pandas as pd

used_models = ["codeparrot/codeparrot-small",
               "EleutherAI/pythia-410m",
               "EleutherAI/pythia-1.4b",
               "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
               "microsoft/phi-2"
               ]

# Assuming 'df' is your DataFrame and you have a specific model ID
model_id = used_models[1]  # Replace with the actual model ID

for model_id in used_models[1:]: 
    # Filter the DataFrame to find the model with the given ID
    model = df[df['id'] == model_id]

    # Get the quartile values for the selected model
    downloads_quartile = model['downloads_quartile']
    created_at_quartile = model['created_at_quartile']
    num_parameters_quartile = model['num_parameters_quartile']

    # Display the quartile values
    print("model: ", model['id'])
    print("Downloads Quartile:", downloads_quartile)
    print("Created At Quartile:", created_at_quartile)
    print("Num Parameters Quartile:", num_parameters_quartile)
    print("---------------------")


model:  1744    EleutherAI/pythia-410m
Name: id, dtype: object
Downloads Quartile: 1744    1
Name: downloads_quartile, dtype: int64
Created At Quartile: 1744    0
Name: created_at_quartile, dtype: int64
Num Parameters Quartile: 1744    0
Name: num_parameters_quartile, dtype: int64
---------------------
model:  208    EleutherAI/pythia-1.4b
Name: id, dtype: object
Downloads Quartile: 208    1
Name: downloads_quartile, dtype: int64
Created At Quartile: 208    0
Name: created_at_quartile, dtype: int64
Num Parameters Quartile: 208    0
Name: num_parameters_quartile, dtype: int64
---------------------
model:  368    TinyLlama/TinyLlama-1.1B-Chat-v1.0
Name: id, dtype: object
Downloads Quartile: 368    1
Name: downloads_quartile, dtype: int64
Created At Quartile: 368    0
Name: created_at_quartile, dtype: int64
Num Parameters Quartile: 368    0
Name: num_parameters_quartile, dtype: int64
---------------------
model:  350    microsoft/phi-2
Name: id, dtype: object
Downloads Quartile: 350    1
